In [1]:
import cPickle
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import operator
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

from pandas_datareader import data, wb
import pandas_datareader as pdr
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
import re
from dateutil import parser
from bt import Strategy#, Portfolio
import quandl

In [2]:
def getStock(symbol, start, end):
    """
    Downloads Stock from Yahoo Finance.
    Computes daily Returns based on Adj Close.
    Returns pandas dataframe.
    """
    
    df =  pdr.data.get_data_yahoo(symbol, start, end)

    df.columns.values[-1] = 'AdjClose'
    df.columns = df.columns + '_' + symbol
    df['Return_%s' %symbol] = df['AdjClose_%s' %symbol].pct_change()
    
    print(df)
    
    return df

d1 = datetime.datetime(2016,5,1)
d2 = datetime.datetime(2017,5,1)

df = getStock("AAPL", d1, d2)

df.columns

             Open_AAPL   High_AAPL    Low_AAPL  Close_AAPL  Volume_AAPL  \
Date                                                                      
2016-05-02   93.970001   94.080002   92.400002   93.639999     48160100   
2016-05-03   94.199997   95.739998   93.680000   95.180000     56831300   
2016-05-04   95.199997   95.900002   93.820000   94.190002     41025500   
2016-05-05   94.000000   94.070000   92.680000   93.239998     35890500   
2016-05-06   93.370003   93.449997   91.849998   92.720001     43458200   
2016-05-09   93.000000   93.769997   92.589996   92.790001     32936400   
2016-05-10   93.330002   93.570000   92.110001   93.419998     33686800   
2016-05-11   93.480003   93.570000   92.459999   92.510002     28719100   
2016-05-12   92.720001   92.779999   89.470001   90.339996     76314700   
2016-05-13   90.000000   91.669998   90.000000   90.519997     44392800   
2016-05-16   92.389999   94.389999   91.650002   93.879997     61259800   
2016-05-17   94.550003   

Index([u'Open_AAPL', u'High_AAPL', u'Low_AAPL', u'Close_AAPL', u'Volume_AAPL',
       u'AdjClose_AAPL', u'Return_AAPL'],
      dtype='object')

In [3]:
def getStockDataFromWeb(fout, start_string, end_string):
    """
    Collects predictors data from Yahoo Finance and Quandl.
    Returns a list of dataframes.
    """
    start = start_string
    end = end_string
    
    sp500 = getStock('^GSPC', start, end)
    nasdaq = getStock('^IXIC', start, end)
    frankfurt = getStock('^GDAXI', start, end)
    london = getStock('^FTSE', start, end)
    paris = getStock('^FCHI', start, end)
    hkong = getStock('^HSI', start, end)
    nikkei = getStock('^N225', start, end)
    australia = getStock('^AXJO', start, end)
    djia = getStock('^DJI', start, end)
    
    out =  pdr.data.get_data_yahoo(fout, start, end)
    out.columns.values[-1] = 'AdjClose'
    out.columns = out.columns + '_Out'
    out['Return_Out'] = out['AdjClose_Out'].pct_change()
    
    return [out, sp500, nasdaq, djia, frankfurt, london, paris, hkong, nikkei, australia]

d1 = datetime.datetime(2016,4,15)
d2 = datetime.datetime(2017,5,1)

bigDF = getStockDataFromWeb('AAPL',d1, d2)

             Open_^GSPC   High_^GSPC    Low_^GSPC  Close_^GSPC  Volume_^GSPC  \
Date                                                                           
2016-04-15  2083.100098  2083.219971  2076.310059  2080.729980    3701450000   
2016-04-18  2078.830078  2094.659912  2073.649902  2094.340088    3316880000   
2016-04-19  2096.050049  2104.050049  2091.679932  2100.800049    3896830000   
2016-04-20  2101.520020  2111.050049  2096.320068  2102.399902    4184880000   
2016-04-21  2102.090088  2103.780029  2088.520020  2091.479980    4175290000   
2016-04-22  2091.489990  2094.320068  2081.199951  2091.580078    3790580000   
2016-04-25  2089.370117  2089.370117  2077.520020  2087.790039    3319740000   
2016-04-26  2089.840088  2096.870117  2085.800049  2091.699951    3557190000   
2016-04-27  2092.330078  2099.889893  2082.310059  2095.149902    4100110000   
2016-04-28  2090.929932  2099.300049  2071.620117  2075.810059    4309840000   
2016-04-29  2071.820068  2073.850098  20

In [4]:
def addFeatures(dataframe, adjclose, returns, n):
    """
    operates on two columns of dataframe:
    - n >= 2
    - given Return_* computes the return of day i respect to day i-n. 
    - given AdjClose_* computes its moving average on n days

    """
    
    return_n = adjclose[9:] + "Time" + str(n)
    dataframe[return_n] = dataframe[adjclose].pct_change(n)
    
    roll_n = returns[7:] + "RolMean" + str(n)
    dataframe[roll_n] = pd.DataFrame(dataframe[returns]).rolling(window = n).mean()



In [5]:
def applyRollMeanDelayedReturns(datasets, delta):
    """
    applies rolling mean and delayed returns to each dataframe in the list
    """
    for dataset in datasets:
        columns = dataset.columns    
        adjclose = columns[-2]
        returns = columns[-1]
        for n in delta:
            addFeatures(dataset, adjclose, returns, n)
    
    return datasets    

delta = {2}

h = applyRollMeanDelayedReturns(bigDF, delta)

In [6]:
h[0]

,Open_Out,High_Out,Low_Out,Close_Out,Volume_Out,AdjClose_Out,Return_Out,OutTime2,OutRolMean2
Date,,,,,,,,,
2016-04-15,112.110001,112.300003,109.730003,109.849998,46939000,107.575818,NaN,NaN,NaN
2016-04-18,108.889999,108.949997,106.940002,107.480003,60834000,105.254888,-0.021575,NaN,NaN
2016-04-19,107.879997,108.000000,106.230003,106.910004,32384900,104.696689,-0.005303,-0.026764,-0.013439
2016-04-20,106.639999,108.089996,106.059998,107.129997,30611000,104.912128,0.002058,-0.003256,-0.001623
2016-04-21,106.930000,106.930000,105.519997,105.970001,31552500,103.776147,-0.010828,-0.008792,-0.004385
2016-04-22,105.010002,106.480003,104.620003,105.680000,33683100,103.492149,-0.002737,-0.013535,-0.006782
2016-04-25,105.000000,105.650002,104.510002,105.080002,28031600,102.904573,-0.005677,-0.008399,-0.004207
2016-04-26,103.910004,105.300003,103.910004,104.349998,56016200,102.189682,-0.006947,-0.012585,-0.006312
2016-04-27,96.000000,98.709999,95.680000,97.820000,114602100,95.794871,-0.062578,-0.069090,-0.034762


In [7]:
def mergeDataframes(datasets, index):
    """
    merges datasets in the list 
    """
    subset = []#tion
    subset = [dataset.iloc[:, index:] for dataset in datasets[1:]]
    
    first = subset[0].join(subset[1:], how = 'outer')
    finance = datasets[0].iloc[:, index:].join(first, how = 'left') 
#    finance = finance[finance.index > cut]
    return finance

index = 6
"""
The cut is not useful
"""
#cut = datetime.datetime(2005,11,4)

mergedDF = mergeDataframes(bigDF, index)

mergedDF

,Return_Out,OutTime2,OutRolMean2,Return_^GSPC,^GSPCTime2,^GSPCRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2016-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-04-18,-0.021575,NaN,NaN,0.006541,NaN,NaN,0.004415,NaN,NaN,0.005962,...,NaN,-0.007270,NaN,NaN,-0.033955,NaN,NaN,-0.003955,NaN,NaN
2016-04-19,-0.005303,-0.026764,-0.013439,0.003084,0.009646,0.004813,-0.003970,0.000427,0.000222,0.002746,...,0.007915,0.012982,0.005617,0.002856,0.036771,0.001568,0.001408,0.010064,0.006069,0.003054
2016-04-20,0.002058,-0.003256,-0.001623,0.000762,0.003848,0.001923,0.001579,-0.002397,-0.001195,0.002364,...,0.009402,-0.009325,0.003535,0.001828,0.001902,0.038744,0.019337,0.005242,0.015359,0.007653
2016-04-21,-0.010828,-0.008792,-0.004385,-0.005194,-0.004436,-0.002216,-0.000453,0.001125,0.000563,-0.006286,...,0.001796,0.018174,0.008679,0.004424,0.027036,0.028989,0.014469,0.010870,0.016170,0.008056
2016-04-22,-0.002737,-0.013535,-0.006782,0.000048,-0.005146,-0.002573,-0.008019,-0.008468,-0.004236,0.001181,...,-0.002427,-0.007178,0.010865,0.005498,0.012029,0.039390,0.019532,-0.006885,0.003911,0.001993
2016-04-25,-0.005677,-0.008399,-0.004207,-0.001812,-0.001764,-0.000882,-0.002128,-0.010130,-0.005073,-0.001472,...,-0.004013,-0.007574,-0.014698,-0.007376,-0.007579,0.004359,0.002225,NaN,NaN,NaN
2016-04-26,-0.006947,-0.012585,-0.006312,0.001873,0.000057,0.000030,-0.001534,-0.003659,-0.001831,0.000728,...,-0.003999,0.004827,-0.002784,-0.001374,-0.004933,-0.012475,-0.006256,-0.003017,-0.009881,-0.004951
2016-04-27,-0.062578,-0.069090,-0.034762,0.001649,0.003525,0.001761,-0.005143,-0.006669,-0.003338,0.002848,...,0.001469,-0.002133,0.002683,0.001347,-0.003618,-0.008533,-0.004275,-0.006302,-0.009300,-0.004660


In [8]:
def applyTimeLag(dataset, lags, delta):
    """
    apply time lag to return columns selected according  to delta.
    Days to lag are contained in the lags list passed as argument.
    Returns a NaN free dataset obtained cutting the lagged dataset
    at head and tail
    """
    
    dataset.Return_Out = dataset.Return_Out.shift(-1)
    
    maxLag = max(lags)
 
    allColumns = dataset.columns[:]
  
    for column in allColumns:
        data_col = pd.Series(dataset[column])
        data_col = data_col.interpolate(method='linear', axis=0, limit=None, inplace=False, limit_direction='forward', downcast=None)
        dataset[column] = data_col

    return dataset.iloc[maxLag:-1,:]

lags = {0, 1, 2}

hhh = applyTimeLag(mergedDF, lags, delta)

hhh

,Return_Out,OutTime2,OutRolMean2,Return_^GSPC,^GSPCTime2,^GSPCRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2016-04-19,0.002058,-0.026764,-0.013439,0.003084,0.009646,0.004813,-0.003970,0.000427,0.000222,0.002746,...,0.007915,0.012982,0.005617,0.002856,0.036771,0.001568,0.001408,0.010064,0.006069,0.003054
2016-04-20,-0.010828,-0.003256,-0.001623,0.000762,0.003848,0.001923,0.001579,-0.002397,-0.001195,0.002364,...,0.009402,-0.009325,0.003535,0.001828,0.001902,0.038744,0.019337,0.005242,0.015359,0.007653
2016-04-21,-0.002737,-0.008792,-0.004385,-0.005194,-0.004436,-0.002216,-0.000453,0.001125,0.000563,-0.006286,...,0.001796,0.018174,0.008679,0.004424,0.027036,0.028989,0.014469,0.010870,0.016170,0.008056
2016-04-22,-0.005677,-0.013535,-0.006782,0.000048,-0.005146,-0.002573,-0.008019,-0.008468,-0.004236,0.001181,...,-0.002427,-0.007178,0.010865,0.005498,0.012029,0.039390,0.019532,-0.006885,0.003911,0.001993
2016-04-25,-0.006947,-0.008399,-0.004207,-0.001812,-0.001764,-0.000882,-0.002128,-0.010130,-0.005073,-0.001472,...,-0.004013,-0.007574,-0.014698,-0.007376,-0.007579,0.004359,0.002225,-0.004951,-0.002985,-0.001479
2016-04-26,-0.062578,-0.012585,-0.006312,0.001873,0.000057,0.000030,-0.001534,-0.003659,-0.001831,0.000728,...,-0.003999,0.004827,-0.002784,-0.001374,-0.004933,-0.012475,-0.006256,-0.003017,-0.009881,-0.004951
2016-04-27,-0.030566,-0.069090,-0.034762,0.001649,0.003525,0.001761,-0.005143,-0.006669,-0.003338,0.002848,...,0.001469,-0.002133,0.002683,0.001347,-0.003618,-0.008533,-0.004275,-0.006302,-0.009300,-0.004660
2016-04-28,-0.011494,-0.091231,-0.046572,-0.009231,-0.007597,-0.003791,-0.011896,-0.016977,-0.008519,-0.011684,...,0.002668,0.001237,-0.000899,-0.000448,-0.036115,-0.039602,-0.019866,0.007267,0.000919,0.000483
2016-04-29,-0.001067,-0.041709,-0.021030,-0.005063,-0.014247,-0.007147,-0.006229,-0.018050,-0.009062,-0.003203,...,-0.014311,-0.015007,-0.013789,-0.006885,-0.033618,-0.052857,-0.026742,0.005129,0.012433,0.006198


In [9]:
def prepareDataForCrossValidation(dataset):
    """
    generate categorical output column, set features and return whole data set
    """
    le = preprocessing.LabelEncoder()
    
    dataset['UpDown'] = dataset['Return_Out']
    dataset.UpDown[dataset.UpDown >= 0] = 'Up'
    dataset.UpDown[dataset.UpDown < 0] = 'Down'
    dataset.UpDown = le.fit(dataset.UpDown).transform(dataset.UpDown)
    """
    Here set feature from column 3 to the end, get rid of all the colomn related to S&P500
    """
    features = dataset.columns[3:-1]
    X = dataset[features]    
    y = dataset.UpDown
    return [X,y]

[X_whole, y_whole] = prepareDataForCrossValidation(hhh)
X_whole

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\pandas\core\generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ljia\Anaco

,Return_^GSPC,^GSPCTime2,^GSPCRolMean2,Return_^IXIC,^IXICTime2,^IXICRolMean2,Return_^DJI,^DJITime2,^DJIRolMean2,Return_^GDAXI,...,^FCHIRolMean2,Return_^HSI,^HSITime2,^HSIRolMean2,Return_^N225,^N225Time2,^N225RolMean2,Return_^AXJO,^AXJOTime2,^AXJORolMean2
Date,,,,,,,,,,,,,,,,,,,,,
2016-04-19,0.003084,0.009646,0.004813,-0.003970,0.000427,0.000222,0.002746,0.008724,0.004354,0.022655,...,0.007915,0.012982,0.005617,0.002856,0.036771,0.001568,0.001408,0.010064,0.006069,0.003054
2016-04-20,0.000762,0.003848,0.001923,0.001579,-0.002397,-0.001195,0.002364,0.005116,0.002555,0.006928,...,0.009402,-0.009325,0.003535,0.001828,0.001902,0.038744,0.019337,0.005242,0.015359,0.007653
2016-04-21,-0.005194,-0.004436,-0.002216,-0.000453,0.001125,0.000563,-0.006286,-0.003937,-0.001961,0.001386,...,0.001796,0.018174,0.008679,0.004424,0.027036,0.028989,0.014469,0.010870,0.016170,0.008056
2016-04-22,0.000048,-0.005146,-0.002573,-0.008019,-0.008468,-0.004236,0.001181,-0.005113,-0.002553,-0.005964,...,-0.002427,-0.007178,0.010865,0.005498,0.012029,0.039390,0.019532,-0.006885,0.003911,0.001993
2016-04-25,-0.001812,-0.001764,-0.000882,-0.002128,-0.010130,-0.005073,-0.001472,-0.000294,-0.000146,-0.007629,...,-0.004013,-0.007574,-0.014698,-0.007376,-0.007579,0.004359,0.002225,-0.004951,-0.002985,-0.001479
2016-04-26,0.001873,0.000057,0.000030,-0.001534,-0.003659,-0.001831,0.000728,-0.000746,-0.000372,-0.003377,...,-0.003999,0.004827,-0.002784,-0.001374,-0.004933,-0.012475,-0.006256,-0.003017,-0.009881,-0.004951
2016-04-27,0.001649,0.003525,0.001761,-0.005143,-0.006669,-0.003338,0.002848,0.003577,0.001788,0.003922,...,0.001469,-0.002133,0.002683,0.001347,-0.003618,-0.008533,-0.004275,-0.006302,-0.009300,-0.004660
2016-04-28,-0.009231,-0.007597,-0.003791,-0.011896,-0.016977,-0.008519,-0.011684,-0.008869,-0.004418,0.002070,...,0.002668,0.001237,-0.000899,-0.000448,-0.036115,-0.039602,-0.019866,0.007267,0.000919,0.000483
2016-04-29,-0.005063,-0.014247,-0.007147,-0.006229,-0.018050,-0.009062,-0.003203,-0.014850,-0.007444,-0.027340,...,-0.014311,-0.015007,-0.013789,-0.006885,-0.033618,-0.052857,-0.026742,0.005129,0.012433,0.006198


In [10]:
def performClassification(X_train, y_train, X_test, y_test, method):
    """
    performs classification on daily returns using several algorithms (method).
    method --> string algorithm
    parameters --> list of parameters passed to the classifier (if any)
    fout --> string with name of stock to be predicted
    savemodel --> boolean. If TRUE saves the model to pickle file
    """
   
    if method == 'RF':   
        return performRFClass(X_train, y_train, X_test, y_test)
        
    elif method == 'KNN':
        return performKNNClass(X_train, y_train, X_test, y_test)
    
    elif method == 'SVM':   
        return performSVMClass(X_train, y_train, X_test, y_test)
    
    elif method == 'ADA':
        return performAdaBoostClass(X_train, y_train, X_test, y_test)
    
    elif method == 'GTB': 
        return performGTBClass(X_train, y_train, X_test, y_test)

    elif method == 'QDA': 
        return performQDAClass(X_train, y_train, X_test, y_test)

In [11]:
def performRFClass(X_train, y_train, X_test, y_test):
    """
    Random Forest Binary Classification
    """
    clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    clf.fit(X_train, y_train)

    feature_importance = clf.feature_importances_
    print feature_importance
    accuracy = clf.score(X_test, y_test)
    
        
    y_predict = clf.predict(X_test)
    
    print("y_predict = ", y_predict)
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print(tn, fp, fn, tp)
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy

#performRFClass (X_train, y_train, X_test, y_test)

In [12]:
def performKNNClass(X_train, y_train, X_test, y_test):
    """
    KNN binary Classification
    """
    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)
            
    y_predict = clf.predict(X_test)    
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    
    return accuracy
#performKNNClass (X_train, y_train, X_test, y_test)

In [13]:
def performSVMClass(X_train, y_train, X_test, y_test):
    """
    SVM binary Classification
    """
    clf = SVC()
    clf.fit(X_train, y_train)

    accuracy = clf.score(X_test, y_test)
    
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy
#performSVMClass (X_train, y_train, X_test, y_test)

In [14]:
def performAdaBoostClass(X_train, y_train, X_test, y_test):
    """
    Ada Boosting binary Classification
    """
    clf = AdaBoostClassifier()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)

    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy
#performAdaBoostClass (X_train, y_train, X_test, y_test)

In [15]:
def performGTBClass(X_train, y_train, X_test, y_test):
    """
    Gradient Tree Boosting binary Classification
    """
    clf = GradientBoostingClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print'(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    accuracy = clf.score(X_test, y_test)
    
    return accuracy
#performGTBClass(X_train, y_train, X_test, y_test)

In [16]:
def performQDAClass(X_train, y_train, X_test, y_test):
    """
    Quadratic Discriminant Analysis binary Classification
    """
   
    clf = QDA()
    clf.fit(X_train, y_train)
    
    accuracy = clf.score(X_test, y_test)
    y_predict = clf.predict(X_test)
    print 'y_predict = ', y_predict
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
    print '(tn, fp, fn, tp) : (', tn, fp, fn,tp , ')'
    
    target_names = ['Down', 'Up']
    print(classification_report(y_test, y_predict, target_names=target_names))
    
    return accuracy

#performQDAClass(X_train, y_train, X_test, y_test)

In [17]:
def timeSeriesCV(dataset_X,dataset_y, n_splits, algorithm):
    print dataset_X.shape
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    counter = 0
    accuracies = np.zeros(n_splits)
    print accuracies
    for train_index, test_index in tscv.split(dataset_X):
        counter += 1
        print 'cyle: ', counter
        print 'TRAIN size: ', train_index[-1], 'TEST size: ', test_index[-1] - test_index[0] 
    
        X_train, X_test = dataset_X[:train_index[-1]], dataset_X[test_index[0]:test_index[-1]]
        y_train, y_test = dataset_y[:train_index[-1]], dataset_y[test_index[0]:test_index[-1]]
        
        print 'X_train shape: ', X_train.shape
        print 'X_test shape: ', X_test.shape
        
        index = counter - 1
        accuracies[index] = performClassification(X_train, y_train, X_test, y_test, algorithm)
        print 'Accuracy: ', accuracies[index], '\n'
        
    print 'Average accuracy: ', np.mean(accuracies)
    return np.mean(accuracies)

    
timeSeriesCV(X_whole, y_whole, 10, 'RF')

(260, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 27)
X_test shape:  (22, 27)
[ 0.02469533  0.01713421  0.02398048  0.03136002  0.04714082  0.04729393
  0.03144904  0.02517182  0.02547576  0.13287408  0.0144983   0.01804863
  0.04841788  0.03855622  0.04520654  0.05959691  0.021049    0.02119005
  0.02437447  0.05072157  0.06006021  0.03300724  0.03712922  0.0406809
  0.02609969  0.02460977  0.03017792]
('y_predict = ', array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1], dtype=int64))
(6, 4, 6, 6)
             precision    recall  f1-score   support

       Down       0.50      0.60      0.55        10
         Up       0.60      0.50      0.55        12

avg / total       0.55      0.55      0.55        22

Accuracy:  0.545454545455 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 27)
X_test shape:  (22, 27)
[ 0.04278459  0.02559593  0.02761426  0.04034908  0.04063607  0.03999943
  0.040

0.48181818181818181

In [18]:
timeSeriesCV(X_whole, y_whole, 10, 'SVM')

(260, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 10 0 12 0 )
             precision    recall  f1-score   support

       Down       0.45      1.00      0.62        10
         Up       0.00      0.00      0.00        12

avg / total       0.21      0.45      0.28        22

Accuracy:  0.454545454545 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 8 0 14 0 )
             precision    recall  f1-score   support

       Down       0.36      1.00      0.53         8
         Up       0.00      0.00      0.00        14

avg / total       0.13      0.36      0.19        22

Accuracy:  0.363636363636 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 27)
X_test shape:  (22, 27)
y_predict = 

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


X_test shape:  (22, 27)
y_predict =  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(tn, fp, fn, tp) : ( 0 9 0 13 )
             precision    recall  f1-score   support

       Down       0.00      0.00      0.00         9
         Up       0.59      1.00      0.74        13

avg / total       0.35      0.59      0.44        22

Accuracy:  0.590909090909 

cyle:  8
TRAIN size:  190 TEST size:  22
X_train shape:  (190, 27)
X_test shape:  (22, 27)
y_predict =  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(tn, fp, fn, tp) : ( 0 6 0 16 )
             precision    recall  f1-score   support

       Down       0.00      0.00      0.00         6
         Up       0.73      1.00      0.84        16

avg / total       0.53      0.73      0.61        22

Accuracy:  0.727272727273 

cyle:  9
TRAIN size:  213 TEST size:  22
X_train shape:  (213, 27)
X_test shape:  (22, 27)
y_predict =  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
(tn, fp, fn, tp) : ( 0 9 0 13 )
             precision    recall  f

0.47727272727272735

In [19]:
timeSeriesCV(X_whole, y_whole, 10, 'ADA')

(260, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0]
(tn, fp, fn, tp) : ( 5 5 6 6 )
             precision    recall  f1-score   support

       Down       0.45      0.50      0.48        10
         Up       0.55      0.50      0.52        12

avg / total       0.50      0.50      0.50        22

Accuracy:  0.5 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 27)
X_test shape:  (22, 27)
y_predict =  [0 1 1 0 1 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1]
(tn, fp, fn, tp) : ( 3 5 7 7 )
             precision    recall  f1-score   support

       Down       0.30      0.38      0.33         8
         Up       0.58      0.50      0.54        14

avg / total       0.48      0.45      0.46        22

Accuracy:  0.454545454545 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 27)
X_test shape:  (22, 27)
y_predict =  [1 1 0 1 0 0 

0.54090909090909089

In [20]:
timeSeriesCV(X_whole, y_whole, 10, 'GTB')

(260, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 0]
(tn, fp, fn, tp) : ( 5 5 5 7 )
             precision    recall  f1-score   support

       Down       0.50      0.50      0.50        10
         Up       0.58      0.58      0.58        12

avg / total       0.55      0.55      0.55        22

Accuracy:  0.545454545455 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 27)
X_test shape:  (22, 27)
y_predict =  [0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 1]
(tn, fp, fn, tp) : ( 6 2 8 6 )
             precision    recall  f1-score   support

       Down       0.43      0.75      0.55         8
         Up       0.75      0.43      0.55        14

avg / total       0.63      0.55      0.55        22

Accuracy:  0.545454545455 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 27)
X_test shape:  (22, 27)
y_predict =  [0

0.49545454545454548

In [21]:
timeSeriesCV(X_whole, y_whole, 10, 'QDA')

(260, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 27)
X_test shape:  (22, 27)
y_predict =  [0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 8 2 10 2 )
             precision    recall  f1-score   support

       Down       0.44      0.80      0.57        10
         Up       0.50      0.17      0.25        12

avg / total       0.47      0.45      0.40        22

Accuracy:  0.454545454545 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(tn, fp, fn, tp) : ( 8 0 14 0 )
             precision    recall  f1-score   support

       Down       0.36      1.00      0.53         8
         Up       0.00      0.00      0.00        14

avg / total       0.13      0.36      0.19        22

Accuracy:  0.363636363636 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 27)
X_test shape:  (22, 27)
y_predict =  

C:\Users\ljia\Anaconda3\envs\py27\lib\site-packages\sklearn\discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


 7 4 6 )
             precision    recall  f1-score   support

       Down       0.56      0.42      0.48        12
         Up       0.46      0.60      0.52        10

avg / total       0.51      0.50      0.50        22

Accuracy:  0.5 

cyle:  4
TRAIN size:  98 TEST size:  22
X_train shape:  (98, 27)
X_test shape:  (22, 27)
y_predict =  [0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0]
(tn, fp, fn, tp) : ( 7 1 9 5 )
             precision    recall  f1-score   support

       Down       0.44      0.88      0.58         8
         Up       0.83      0.36      0.50        14

avg / total       0.69      0.55      0.53        22

Accuracy:  0.545454545455 

cyle:  5
TRAIN size:  121 TEST size:  22
X_train shape:  (121, 27)
X_test shape:  (22, 27)
y_predict =  [0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1]
(tn, fp, fn, tp) : ( 6 9 5 2 )
             precision    recall  f1-score   support

       Down       0.55      0.40      0.46        15
         Up       0.18      0.29      0.22       

0.43636363636363634

In [22]:
timeSeriesCV(X_whole, y_whole, 10, 'KNN')

(260, 27)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
cyle:  1
TRAIN size:  29 TEST size:  22
X_train shape:  (29, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1]
(tn, fp, fn, tp) : ( 8 2 10 2 )
             precision    recall  f1-score   support

       Down       0.44      0.80      0.57        10
         Up       0.50      0.17      0.25        12

avg / total       0.47      0.45      0.40        22

Accuracy:  0.454545454545 

cyle:  2
TRAIN size:  52 TEST size:  22
X_train shape:  (52, 27)
X_test shape:  (22, 27)
y_predict =  [0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0]
(tn, fp, fn, tp) : ( 6 2 10 4 )
             precision    recall  f1-score   support

       Down       0.38      0.75      0.50         8
         Up       0.67      0.29      0.40        14

avg / total       0.56      0.45      0.44        22

Accuracy:  0.454545454545 

cyle:  3
TRAIN size:  75 TEST size:  22
X_train shape:  (75, 27)
X_test shape:  (22, 27)
y_predict =  

0.47727272727272735